#  Los he ordenado por similaritud, me he estado liando un poco con lo de los excells, os dejo los dos formatos, no se cual os gustara mas, pero vaya qye si cinseguis juntarlos bien.

In [ ]:
# me ha hecho falta instalar esto
pip install xlsxwriter

## este es el primero, que deja la query oero el nombre de las columnas no

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Read data from Excel file into a dictionary of DataFrames (one for each sheet)
file_path = 'data/loinc_dataset-v2.xlsx'
excel_data = pd.read_excel(file_path, sheet_name=None, skiprows=2)

# Create a new Excel file to store the ranked results
output_file_path = 'data/ranked_loinc_dataset_same_format_multi_sheets.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    # Iterate through each sheet
    for sheet_name, df in excel_data.items():
        # Drop empty rows
        df = df.dropna()

        # Extract features for the query row
        query_row = df.iloc[0]  # Assuming the first row is the query
        query_features = query_row[['loinc_num', 'long_common_name', 'component', 'system']].astype(str)

        # Combine the text columns into a single string for vectorization
        query_text = ' '.join(query_features.values)

        # Vectorize the text using TF-IDF for the query
        vectorizer = TfidfVectorizer()
        query_vectorized = vectorizer.fit_transform([query_text])

        # Initialize a list to store similarity scores
        similarity_scores = []

        # Iterate through each data row until the next empty row and calculate similarity
        for index, row in df.iterrows():
            # Check for an empty row
            if row.isnull().all():
                break

            data_features = row[['loinc_num', 'long_common_name', 'component', 'system']].astype(str)
            data_text = ' '.join(data_features.values)

            # Vectorize the text using TF-IDF for the data row
            data_vectorized = vectorizer.transform([data_text])

            # Calculate cosine similarity
            similarity_score = cosine_similarity(query_vectorized, data_vectorized)[0][0]

            # Append similarity score to the list
            similarity_scores.append(similarity_score)

        # Add similarity scores to the DataFrame
        df['Similarity'] = similarity_scores

        # Sort DataFrame by similarity in descending order
        ranked_df = df.sort_values(by='Similarity', ascending=False)

        # Create a new DataFrame with the required format
        result_df = pd.DataFrame(columns=ranked_df.columns)
        result_df.loc[0] = [f'Query: {sheet_name}'] + [''] * (result_df.shape[1] - 1)
        result_df.loc[1] = [''] * result_df.shape[1]

        # Concatenate the new DataFrame with the column names and the ranked DataFrame
        result_df = pd.concat([result_df, ranked_df], ignore_index=True)

        # Save the result DataFrame to the Excel file with the original sheet name
        result_df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

# Print a message indicating that the DataFrame has been saved
print(f"The final ranked DataFrame has been saved to {output_file_path}")


   loinc_num                                   long_common_name  \
0     1988-5  C reactive protein [Mass/volume] in Serum or P...   
43   30522-7  C reactive protein [Mass/volume] in Serum or P...   
10    4671-4                  Protein C [Mass/volume] in Plasma   
8     2143-6          Cortisol [Mass/volume] in Serum or Plasma   
21    1968-7  Bilirubin.direct [Mass/volume] in Serum or Plasma   
..       ...                                                ...   
11   18864-9                        Ampicillin [Susceptibility]   
24    8310-5                                   Body temperature   
17     925-8                   Blood product disposition [Type]   
48   18955-5                    Nitrofurantoin [Susceptibility]   
66   23658-8                  Other Antibiotic [Susceptibility]   

                                           component    system  Similarity  
0                                 C reactive protein  Ser/Plas    1.000000  
43                                C react

## Y este es el segundo, que deja las columnas pero no el nombre de la query

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Read data from Excel file into a dictionary of DataFrames (one for each sheet)
file_path = 'data/loinc_dataset-v2.xlsx'
excel_data = pd.read_excel(file_path, sheet_name=None, skiprows=2)

# Create a new Excel file to store the ranked results
output_file_path = 'data/ranked_loinc_dataset_same_format_multi_sheets.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    # Iterate through each sheet
    for sheet_name, df in excel_data.items():
        # Drop empty rows
        df = df.dropna()

        # Extract features for the query row
        query_row = df.iloc[0]  # Assuming the first row is the query
        query_features = query_row[['loinc_num', 'long_common_name', 'component', 'system']].astype(str)

        # Combine the text columns into a single string for vectorization
        query_text = ' '.join(query_features.values)

        # Vectorize the text using TF-IDF for the query
        vectorizer = TfidfVectorizer()
        query_vectorized = vectorizer.fit_transform([query_text])

        # Initialize a list to store similarity scores
        similarity_scores = []

        # Iterate through each data row until the next empty row and calculate similarity
        for index, row in df.iterrows():
            # Check for an empty row
            if row.isnull().all():
                break

            data_features = row[['loinc_num', 'long_common_name', 'component', 'system']].astype(str)
            data_text = ' '.join(data_features.values)

            # Vectorize the text using TF-IDF for the data row
            data_vectorized = vectorizer.transform([data_text])

            # Calculate cosine similarity
            similarity_score = cosine_similarity(query_vectorized, data_vectorized)[0][0]

            # Append similarity score to the list
            similarity_scores.append(similarity_score)

        # Add similarity scores to the DataFrame
        df['Similarity'] = similarity_scores

        # Sort DataFrame by similarity in descending order
        ranked_df = df.sort_values(by='Similarity', ascending=False)

        # Create a new DataFrame with the required format
        result_df = pd.DataFrame(columns=ranked_df.columns)
        result_df.loc[0] = [None] * (result_df.shape[1])  # Initialize with None
        result_df.loc[0, 'loinc_num'] = 'loinc_num'
        result_df.loc[0, 'long_common_name'] = 'long_common_name'
        result_df.loc[0, 'component'] = 'component'
        result_df.loc[0, 'system'] = 'system'
        result_df.loc[0, 'Similarity'] = 'Similarity'

        # Concatenate the new DataFrame with the column names and the ranked DataFrame
        result_df = pd.concat([result_df, ranked_df], ignore_index=True)

        # Save the result DataFrame to the Excel file with the original sheet name
        result_df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

# Print a message indicating that the DataFrame has been saved
print(f"The final ranked DataFrame has been saved to {output_file_path}")


# Esto no esta bien, es aplicando lo de la regresion pero vaya qye no esta.

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Read data from Excel file into a DataFrame, skipping the empty row and starting from the third row
file_path = 'loinc_dataset-v2.xlsx'
df = pd.read_excel(file_path, skiprows=2)
df = df.dropna()

# Extract features for the query row
query_row = df.iloc[0]  # Assuming the first row is the query
query_features = query_row[['loinc_num', 'long_common_name', 'component', 'system']].astype(str)

# Combine the text columns into a single string for vectorization
query_text = ' '.join(query_features.values)

# Vectorize the text using TF-IDF for the query
vectorizer = TfidfVectorizer()
query_vectorized = vectorizer.fit_transform([query_text])

# Initialize a list to store logistic regression scores
logistic_scores = []

# Initialize logistic regression model
logistic_model = LogisticRegression()

# Iterate through each data row until the next empty row and apply logistic regression
for index, row in df.iterrows():
    # Check for an empty row
    if row.isnull().all():
        break
    
    data_features = row[['loinc_num', 'long_common_name', 'component', 'system']].astype(str)
    data_text = ' '.join(data_features.values)
    
    # Vectorize the text using TF-IDF for the data row
    data_vectorized = vectorizer.transform([data_text])
    
    # Use logistic regression to predict rank (binary classification)
    logistic_score = logistic_model.fit(query_vectorized, [1]*len(data_vectorized)).predict_proba(data_vectorized)[:, 1]
    
    # Append logistic score to the list
    logistic_scores.append(logistic_score[0])

# Add logistic scores to the DataFrame
df['Logistic_Score'] = logistic_scores

# Sort DataFrame by logistic score in descending order
r


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

ValueError: cannot set a row with mismatched columns

  Obtaining dependency information for xlsxwriter from https://files.pythonhosted.org/packages/a7/ea/53d1fe468e63e092cf16e2c18d16f50c29851242f9dd12d6a66e0d7f0d02/XlsxWriter-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   ------- -------------------------------- 30.7/159.9 kB ? eta -:--:--
   --------- ----------------------------- 41.0/159.9 kB 330.3 kB/s eta 0:00:01
   ---------------------- ---------------- 92.2/159.9 kB 751.6 kB/s eta 0:00:01
   ---------------------- ---------------- 92.2/159.9 kB 751.6 kB/s eta 0:00:01
   -------------------------------------- 159.9/159.9 kB 686.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


The final ranked DataFrame has been saved to data/ranked_loinc_dataset_same_format_multi_sheets.xlsx
